## Streamline Solving+Combination code

- Inital solving class has been cut down to have method for finding fundamental conductance matrix (WA)
- 

In [2]:
# Imports

import hypernetx as hnx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import warnings 

warnings.simplefilter('ignore')

from sympy import *

init_printing()

## Class for finding fundamental CM

In [ ]:

class FundamentalConductance:

    """ Class to find the fundamental conductane matrix of an inputted stoichiometric matrix
    
    Attributes:
        SM (Sympy Matrix): Stoichiometric Matrix of the reaction module
        num_internal_species (int): Number of internal species in the reaction module
        
        cycle_matrix: The reaction level cycles matrix for internal species
        coupling_matrix: The coupling matrix between internal and external species
        chemostat_laws(): The conservation law matrices for the full stoichiometric matrix and the chemostat species
        cycle_resistance_matrix: The reaction level resistance matrix for the module with sympy labelling of reactions
        physical_conductance_matrix: The physical level conductance matrix

    Methods:
        calculate_fundamental_conductance_matrix(): Computes the fundamental conductance matrix,
          simplified CM, and determinant of cycle resistance matrix
    """
    
    #==========================================================================================================================================
    # INIT
    #

    def __init__(self, SM, num_internal_species):

        """ Initializes the ModuleProperties class with the full stoichiometric matrix, the internal and external stoichiometric matrices,
        and the labels of the species, reactions, forces, chemical potentials."""

        self.matrix = SM # Returns the passed SM, ready for print

        self.internal_SM = SM[0:num_internal_species, :] # finds the internal species SM by selecting the number of rows needed

        self.external_SM = SM[num_internal_species: len(SM), :] # finds SM for external species using remaning rows after internal species
        
        # LABELS FOR ALL RESISTANCES AND REACTIONS

        num_cols = self.matrix.cols # Finds number of columns ( == no. of reactions) in the SM
       
        resistances = [] # define list to hold reaction labels


        for n in range(num_cols): # loop over each reaction

            nth_resistance = symbols(f"r{n+1}") # assign name of nth resistance
            resistances.append(nth_resistance) # add to list of resistance

    #==========================================================================================================================================
    # REACTION LEVEL CYCLES
    #


        reaction_cycles = (self.internal_SM).nullspace() # finds the kernel for the SM internal

        # Check if there are any cycles:

        if not reaction_cycles:

            print("No internal cycles. Kernel is empty.")

        # build cycle matrix from kernel vectors if kernel is NOT empty

        else:

            cycle_matrix = reaction_cycles[0] # add first vector to cycle matrix so we can add rest later
            
            for cycle in reaction_cycles[1:]: # starting at second vector in kernel

                cycle_matrix = cycle_matrix.row_join(cycle) # connect vectors from kernel column-wise, row_join puts elemetns of adjacent vectors together


            self.cycle_matrix = cycle_matrix # assign cycle matrix to self for use in other methods
            
        #==========================================================================================================================================
        # COUPLING MATRICES
        #  

        phi = self.external_SM * self.cycle_matrix

        self.coupling_matrix = phi
    
        #==========================================================================================================================================
        # CONSERVATION LAW MATRICES
        #

        cokernel_SM = (self.matrix.T).nullspace() # finds the cokernel of the full SM

        if not cokernel_SM:

            print("No conservation laws. Cokernel of Stoichiometric Matrix empty.")

        else:

            cons_laws = cokernel_SM[0] # adds first element of cokernel

            for vec in cokernel_SM[1:]: # add vectors from next row onwards

                cons_laws = cons_laws.row_join(vec)


        #
        # Broken external laws for chemostat , deriving from the coupling matrix
        #

        cokernel_coupling_matrix = self.coupling_matrix.T.nullspace() # find the cokernel of the coupling matrix

        if not cokernel_coupling_matrix:

            print("No chemostat conservation laws. Cokernel of Coupling Matrix is empty.")

        # if cokernel is NOT empty

        else:

            chemostat_laws = cokernel_coupling_matrix[0] # add first vector to chemostat conservation law matrix so we can add rest later

            for law in cokernel_coupling_matrix[1:]: # starting at second vector in kernel

                chemostat_laws = chemostat_laws.row_join(law) # connect vectors from kernel column-wise, row_join puts elemetns of adjacent vectors together

        self.chemostat_laws = chemostat_laws.T # assign to self for use in other methods
        
        #==========================================================================================================================================
        # REACTION LEVEL RESISTANCE MATRIX
        #

        num_cols = self.matrix.cols # Finds number of columns ( == no. of reactions) in the SM

        resistances = [] # define list to hold reaction labels

        for n in range(num_cols):

            nth_reaction = symbols(f"r{n+1}") # assign name of nth reaction

            resistances.append(nth_reaction) # add to list of reactions



        resistance_matrix = Matrix.diag(resistances) # create diagonal matrix from list of reactions

        self.reaction_resistance_matrix = resistance_matrix # assign to self for use in other methods

        #==========================================================================================================================================
        # CYCLE RESISTANCE MATRIX
        #

        cycle_resistance_matrix = self.cycle_matrix.T * self.reaction_resistance_matrix \
            * self.cycle_matrix
        
        self.cycle_resistance_matrix = cycle_resistance_matrix

        #==========================================================================================================================================
        # CONDUCATANCE MATRICES
        #

        physical_conductance_matrix = self.coupling_matrix * self.cycle_resistance_matrix.inv() \
            * self.coupling_matrix.T 

        self.physical_conductance_matrix = physical_conductance_matrix       

    #==========================================================================================================================================
    #   FUNDAMENTAL CONDUCTANCE MATRIX
    #
    
    def calculate_fundamental_conductance_matrix(self):

        """Calculats the fundamental conductance matrix for the module, building the selection matrix from the kernel of the chemostat conser
        -vation laws, finding the pseudoinverse for this matrix and using the physical conductance matrix
        
        Returns:
            fundamental_CM (Sympy Matrix): Fundamental conductance matrix for the module"""
        

        cons_laws_kernel = self.chemostat_laws.nullspace() 

        
        if not cons_laws_kernel:

            print("No selection matrix calculated. Kernel of chemostat conservation laws matrix is empty.")
    
        result = cons_laws_kernel[0]

        for i in range(1,len(cons_laws_kernel)):
            
            result = result.row_join(-1*cons_laws_kernel[i]) # *-1 here to put matter flow in correct direction

        selection_matrix = result

        selection_matrix_MPI = (selection_matrix.T*selection_matrix).inv() * selection_matrix.T

        

        fundamental_CM = selection_matrix_MPI * self.physical_conductance_matrix * selection_matrix_MPI.T

        if fundamental_CM.shape == (1,1): # if matrix is a single element - do not want to simplify it

            return fundamental_CM
        
        else: # if matrix is not a single element: output the full CM, the simplified CM (multiplied by det(R)) and the factor 1/det(R)

            return fundamental_CM, fundamental_CM * self.cycle_resistance_matrix.det(), 1/self.cycle_resistance_matrix.det()
        
    #==========================================================================================================================================


# Test Space

In [41]:
stoich_matrix_2 = Matrix([[-1, 0, 1, 0, 0], # first row of internal
                           [1, -1, 0, -1, 0], 
                           [0, 1, -1, 0, 1], 
                           [0, 0, 0, 1, -1],
                             [0, 0, 0, -1, 0], # first row external 
                             [0, 0, 0, 0, 1],
                             [-1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0]])



mod2 = FundamentalConductance(stoich_matrix_2, 4)

mod2.calculate_fundamental_conductance_matrix()[1]

⎡r₁ + r₂ + r₃      -r₂     ⎤
⎢                          ⎥
⎣    -r₂       r₂ + r₄ + r₅⎦

## Class for combination: